In [ ]:
import lightkurve as lk
import pandas as pd
import pickle
import emcee
import corner
import speclite as speclite; from speclite import filters
from tqdm import tqdm
from matplotlib import cm
from matplotlib.artist import Artist
from chromatic import *
from scipy.optimize import minimize
from scipy.optimize import curve_fit

# params = {'legend.fontsize': 'medium',
#           'figure.figsize': (6, 4),
#          'axes.labelsize': 'large',
#          'axes.titlesize':'x-large',
#          'xtick.labelsize':'large',
#          'ytick.labelsize':'large'}
# plt.rcParams.update(params)
# plt.style.use('tableau-colorblind10')
# plt.rcParams['lines.linewidth']   =3
# plt.rcParams['axes.linewidth']    = 2
# plt.rcParams['xtick.major.width'] = 6
# plt.rcParams['ytick.major.width'] = 6
# plt.rcParams['ytick.labelsize'] = 12
# plt.rcParams['xtick.labelsize'] = 12
# plt.rcParams['axes.labelsize'] = 15
# plt.rcParams['legend.numpoints'] = 1
# plt.rcParams['axes.labelweight']='semibold'
# plt.rcParams['mathtext.fontset']='stix'
# plt.rcParams['font.weight'] = 'semibold'
# plt.rcParams['axes.titleweight']='semibold'
# plt.rcParams['axes.titlesize']=9

In [ ]:
def recreate_lc(visit,filt,camera):
    
    t = np.load(f'../data/LCO_Photometry/{visit}_{filt}_{camera}_times.npy')
    f = np.load(f'../data/LCO_Photometry/{visit}_{filt}_{camera}_fluxes.npy')
    e = np.load(f'../data/LCO_Photometry/{visit}_{filt}_{camera}_uncertainties.npy')
    
    newlc = lk.LightCurve(time = t*u.day, flux = f, flux_err = e)

    return newlc

In [ ]:
def sinusoid_model(time, parameters,Period = 4.865):
        
    A = parameters[0]
    B = parameters[1]
    
    argument = (2.0 * np.pi * time)/Period
    
    model = A*np.sin(argument) + B*np.cos(argument) + 1
    
    return model

In [ ]:
visits = ['F21','S22']

fig,[ax21,ax22] = plt.subplots(2,1,figsize=(12,5))

for visit in tqdm(visits):
    
    ax21.set_title(f'F21 Data',fontsize=15)
    ax22.set_title(f'S22 Data',fontsize=15)
        
    if visit == 'F21':
        xlims=[2459447,2459469]
        filt_names=['gp','rp','ip']
        labels = ['LCO Siding Spring', 'LCO CTIO', 'LCO Teide', 'LCO Siding Spring']
        cam_names = ['kb24', 'kb26', 'kb29', 'kb88']
        colors = ['#fac205','#0165fc','#8c000f','#ed0dd9']
        T0 = (59455.48 + 2400000.5)
        ndim = 6  
        ncams = 4
        spectimes = [2459451.5,2459453.,2459455.5,2459458.,2459459.5,2459461.5,2459465.]
        ax = ax21
        
    if visit == 'S22':
        xlims=[2459671,2459698]
        filt_names=['gp','rp']
        labels = ['LCO Siding Spring', 'LCO CTIO', 'LCO Teide', 'LCO Sutherland', 'LCO Siding Spring']
        colors = ['#fac205','#0165fc','#8c000f','#10a674','#ed0dd9']
        cam_names = ['kb24', 'kb26', 'kb29', 'kb87', 'kb88']
        T0 = (59455.48 + 2400000.5) + (27.*8.4631427)
        ndim = 7  
        ncams = 5
        spectimes = [2459672.,2459673.,2459674.,2459675.,2459676.,2459677.,
                     2459678.,2459679.,2459680.,2459681.,2459682.,2459683.,2459684.,2459685.,2459686.,2459687.,2459688.]
        ax = ax22
        
    for this_filter in tqdm(filt_names):
        
        if this_filter =='gp':
            this_filt_color='orange'
        if this_filter=='rp':
            this_filt_color='teal'
        if this_filter=='ip':
            this_filt_color='purple'

        fitted_parameters = np.load(f'../data/rainbows/{visit}_{this_filter}_fittedparameters.npy')
        clipped_lcs = [None]*ncams 
        for m in range(0,ncams):
            clipped_lcs[m] = recreate_lc(visit=visit,filt=this_filter,camera=cam_names[m])

        hires_model_times = np.linspace(xlims[0],xlims[1],2000)

        # Plot a high-resolution model from the best fit
        # best_model = sinusoid_model(hires_model_times, fitted_parameters)
        # ax.plot(hires_model_times, best_model, c='k',zorder=-100,linewidth=2)
        ax.set_ylabel('Relative Flux',fontsize=15)
        # ax.set_ylim(0.9,1.15)
        ax.set_xlim(T0-15, T0+15)
        ax22.set_xlabel(f'Time [BJD]',fontsize=15)
        
        j=0
        for spectime in spectimes:
            
            if j==0:
                ax.axvline(spectime,label='NRES Observation',color='green',linewidth=1,alpha=0.7)
            else:
                ax.axvline(spectime,color='green',linewidth=1,alpha=0.7)
            j+=1

        
        # Plot our 4 different data sets in this filter
        for i in range(0,ncams):
            this_lc = clipped_lcs[i]
            this_lc.flux = clipped_lcs[i].flux - fitted_parameters[i+2]
            data_model = sinusoid_model(this_lc.time.value,fitted_parameters)
            # Plot the data
            ax.errorbar(this_lc.time.value,this_lc.flux,yerr=this_lc.flux_err,
                         label=labels[i],color=colors[i],alpha=0.8,zorder=100,linewidth=0,elinewidth=2)
        
        ax.axvline(T0,label='AU Mic b Transit',color='red',linewidth=2,alpha=0.7)
        if this_filter == 'gp':
            ax.legend(loc='upper left',fontsize=10,framealpha=1)

plt.savefig(f'../figs/data_exposition.png',dpi=300)
plt.show()